## Setup danych (nieistonte)

In [35]:
# from google.colab import files
# uploaded = files.upload()

In [36]:
import pandas
import numpy as np
import os

In [37]:
path = 'dane.xls'
dataframe = pandas.read_excel(path, header = 9, usecols='K:AM,AT:BF')

In [38]:
pl_eng_dict = {'Nr źródła': 'Id',
 'Wydzielenia grafitu [mm-2]': 'Graphite emission [mm-2]',
 'Udział wydzieleń grafitu [%]': 'Graphite emission share [%]',
 'Średnica sferoidów [μm]': "Spheroid diameter [μm]",
 'Wielkość sferoidów': 'Spheroid size',
 'Udział perlitu [%]': 'Perlite share [%]',
 'Udział ferrytu [%]': 'Ferrite share [%]',
 'Minimalna grubość ścianki [mm]': 'Min wall thickness [mm]',
 'Temperatura austenityzacji [˚C]': 'Austenitization temp [C]',
 'Czas austenityzacji [min.]': 'Austenitization time [min.]',
 'Temperatura przemiany izotermicznej [˚C]': 'Isothermal process temp [C]',
 'Czas przemiany izotermicznej [min.]': 'Isothermal process time [min.]',
 'Twardość Brinella [HB]': 'HB',
 'Twardość Rockwella [HRC]': 'HRC',
 'Twardość Rockwella [HRA]': 'HRA',
 'Twardość Rockwella [HRB]': 'HRB',
 'Twardość Vickersa [HV]': 'HV',
 'Udarność Charpy [J]': 'Charpy impact strength [J]',
 'Temperatura pomiaru udarności [˚C]': 'Impact test temp [C]',
 'Udział austenitu %': 'Austenite share [%]',
 "Martensite volume fraction Xα'": 'Martensite volume',
 'Retained austenite volume fraction XγR ': 'Retained austenite volume'}

In [39]:
dataframe.rename(columns=pl_eng_dict, inplace=True)

In [40]:
df_X, df_y = dataframe.iloc[:, 0:29].copy(), dataframe.iloc[:, 29:].copy()

In [41]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rm [MPa]                    1094 non-null   float64
 1   R0,2 [MPa]                  844 non-null    float64
 2   A5 [%]                      968 non-null    float64
 3   HB                          416 non-null    float64
 4   HRC                         435 non-null    float64
 5   HRA                         12 non-null     float64
 6   HRB                         0 non-null      float64
 7   HV                          188 non-null    float64
 8   Charpy impact strength [J]  338 non-null    float64
 9   Impact test temp [C]        312 non-null    float64
 10  Austenite share [%]         295 non-null    float64
 11  Martensite volume           49 non-null     float64
 12  Retained austenite volume   212 non-null    float64
dtypes: float64(13)
memory usage: 153.

In [42]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Id                              1505 non-null   object 
 1   C [%]                           1441 non-null   float64
 2   Si [%]                          1441 non-null   float64
 3   S [%]                           1377 non-null   float64
 4   P [%]                           1362 non-null   float64
 5   Mg [%]                          1278 non-null   float64
 6   Mn [%]                          1402 non-null   float64
 7   Ni [%]                          1261 non-null   float64
 8   Cu [%]                          1225 non-null   float64
 9   Mo [%]                          1062 non-null   float64
 10  Cr [%]                          475 non-null    float64
 11  Al [%]                          413 non-null    float64
 12  Sn [%]                          41

## Bezmyślne wywalenie NaN i inne glupie transformacje (nieistonte)

In [43]:
df_y = df_y.drop('HRB', axis=1)

In [44]:
from sklearn.impute import SimpleImputer

In [45]:
len(df_y)

1508

In [46]:
bad_id = ["106/103", "52/80", "55/77", "84/48"]
bad_id

['106/103', '52/80', '55/77', '84/48']

In [47]:
for i in bad_id:
    df_X = df_X[df_X.Id != i]

df_X = df_X[df_X['Spheroid size'] != '6 i 7']
df_y = df_y.iloc[range(len(df_X)),:]

In [48]:
imp = SimpleImputer(strategy="most_frequent")
df_X[:] = imp.fit_transform(df_X)
df_y[:] = imp.fit_transform(df_y)

<ipython-input-48-d5979864929b>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_X[:] = imp.fit_transform(df_X)
<ipython-input-48-d5979864929b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y[:] = imp.fit_transform(df_y)
<ipython-input-48-d5979864929b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y[:] = imp.fit_transform(df_

In [49]:
X_train = df_X.copy()
y_train = df_y.copy()

## Model do wyboru zmiennych

In [50]:
# !pip install xgboost
# !pip install boruta

In [51]:
import xgboost as xgb
from boruta import BorutaPy

In [52]:
model = xgb.XGBRegressor()

# Wazne informacje:


*   X_train i y_train musza miec ta liczbe wierszy
*   X_train i y_train nie moga miec brakow danych
*   "106/103", "52/80", "55/77", "84/48" tych wartosci trzeba sie pozbyc z kolumny Id lub je w jakis inny sposob zapisac
*   analogicznie dla '6 i 7' ze Spheroid size


Kod ponizej dla kazdej zmiennej odpowiedzi wybiera, ktore zmienne sa istotne podczas jej przewidywania.
Mozna by go usprawnic podajac ramki danych recznie, dla kazdej zmiennej odpowiedzi.

To jest kod jesli bedziemy chcieli dla kazdej zmiennej odpowiedzi dopasowywac osobny model.



In [53]:
feat_selector = BorutaPy(model, n_estimators='auto', random_state=0, verbose=0)

In [54]:
names = df_X.columns

In [55]:
feats = {}
unsure = {}
for i in y_train.columns:
  feat_selector.fit(X_train.values, y_train[i].values)
  accept = names[feat_selector.support_].to_list()
  feats.update({i : accept})
  unsure.update({i : names[feat_selector.support_weak_].to_list()})

In [56]:
feats

{'Rm [MPa]': ['Id',
  'S [%]',
  'P [%]',
  'Mg [%]',
  'Mn [%]',
  'Ni [%]',
  'Mo [%]',
  'Cr [%]',
  'Al [%]',
  'V [%]',
  'Spheroid diameter [μm]',
  'Perlite share [%]',
  'Rm [MPa] (as cast)',
  'Austenitization temp [C]',
  'Isothermal process temp [C]'],
 'R0,2 [MPa]': ['Id',
  'Mn [%]',
  'Ni [%]',
  'Mo [%]',
  'Cr [%]',
  'V [%]',
  'Graphite emission [mm-2]',
  'Spheroid diameter [μm]',
  'Perlite share [%]',
  'Rm [MPa] (as cast)',
  'A5 [%]     (as cast)',
  'Min wall thickness [mm]',
  'Austenitization temp [C]',
  'Isothermal process temp [C]'],
 'A5 [%]': ['Id',
  'P [%]',
  'Mg [%]',
  'Mn [%]',
  'Al [%]',
  'Sn [%]',
  'Graphite emission [mm-2]',
  'Spheroid diameter [μm]',
  'Perlite share [%]',
  'Rm [MPa] (as cast)',
  'A5 [%]     (as cast)',
  'Min wall thickness [mm]',
  'Austenitization temp [C]',
  'Austenitization time [min.]',
  'Isothermal process temp [C]'],
 'HB': ['Id',
  'P [%]',
  'Mg [%]',
  'Mn [%]',
  'Mo [%]',
  'Rm [MPa] (as cast)',
  'A5 [%]   